In [1]:
import pandas as pd
import numpy as np

import torch
import torch.optim as optim
import torch.nn.functional as F

torch.manual_seed(1)

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submit= pd.read_csv('sample.csv')

In [3]:
train

,N,P,K,temperature,humidity,ph,rainfall,label
0,22,36,16,30.581395,50.771481,8.184229,64.585596,mothbeans
1,18,27,41,22.365094,92.308824,7.175344,104.821633,pomegranate
2,61,68,50,35.214628,91.497251,6.793245,243.074507,papaya
3,26,72,22,28.767949,37.577921,4.674942,91.720849,pigeonpeas
4,2,140,197,22.697801,92.822234,5.534567,105.050823,apple
...,...,...,...,...,...,...,...,...
1645,10,5,5,21.213070,91.353492,7.817846,112.983436,orange
1646,108,94,47,27.359116,84.546250,6.387431,90.812505,banana
1647,11,36,31,27.920633,51.779659,6.475449,100.258567,mango
1648,11,124,204,13.429886,80.066340,6.361141,71.400430,grapes


In [4]:
train_x = train.drop(['label'], axis =1)
test_x = test
classle = LabelEncoder()

scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

train.label = classle.fit_transform(train.label)
train_y = pd.DataFrame(train.label)


In [5]:
train.label.unique().max()

21

In [6]:
train_x_tensor = torch.FloatTensor(train_x) #1650,7
train_y_tensor = torch.LongTensor(train_y.values) #1650,1

test_x_tensor = torch.FloatTensor(test_x)

In [7]:
W = torch.zeros((7,22), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

In [8]:
optimizer = optim.Adam([W,b], lr=0.01)
total_epoch= 5000
for epoch in range(1,total_epoch+1):
    hypothesis = F.softmax(train_x_tensor.matmul(W) + b, dim = 1)
    
    y_one_hot = torch.zeros(len(train.label), 22)
    y_one_hot.scatter_(1, train_y_tensor, 1)
    
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 1000 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, total_epoch, cost.item()
        ))

Epoch 1000/5000 Cost: 2.271437
Epoch 2000/5000 Cost: 2.228666
Epoch 3000/5000 Cost: 2.213967
Epoch 4000/5000 Cost: 2.206140
Epoch 5000/5000 Cost: 2.201037


In [9]:
train_hypothesis = F.softmax(train_x_tensor.matmul(W) + b, dim = 1)
predict_train = torch.argmax(train_hypothesis, dim = 1)
print(accuracy_score(train.label, predict_train))
predict_hypothesis = F.softmax(test_x_tensor.matmul(W) + b, dim = 1)
predict = torch.argmax(predict_hypothesis, dim = 1)

0.9775757575757575


In [10]:
submit.label = classle.inverse_transform(predict.detach().numpy())

In [11]:
submit.to_csv('2021_09_29_04.csv', index=False) #0.932727